WE WANT:
* Player (DONE, player.csv)
* GMs
* Game (PARTIAL, game_partial.csv)
* Player-Game
* Alignments (PARTIAL, alignment.csv, just copied from key)
* Deaths (PARTIAL, death.csv, just copied from key)

In [122]:
import pandas as pd 

data = pd.read_csv("original_data.csv")

data.rename(inplace=True, columns={
    "Player": "player_name",
    "Alignment": "alignment_type",
    "Faction Outcome": "win", 
    "First Hit": "first_hit", 
    "Last Hit": "last_hit",
    "# of Hits": "num_hits",
    "Death/\r\n Survival": "death", # ???
    "Inactive": "inactive",
    "Mod": "IM",
    "Game Type": "format",
    "Game #": "game_num",
    "Unnamed: 14": "game_string",
    "Role": "role",
    "Unnamed: 17": "secondary_role"
    })

data.head()

,ID,player_name,alignment_type,win,first_hit,last_hit,num_hits,death,inactive,GM,IM,Spec,format,game_num,game_string,Broken,role,secondary_role
0,1,Meta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,LG,1.0,LG1,NaN,GM,NaN
1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Tineye,NaN
2,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Cop,NaN
3,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Regular,NaN
4,5,Aether,E,W,4.0,4.0,1.0,V,NaN,NaN,NaN,NaN,LG,1.0,LG1,NaN,Smoker,NaN


In [108]:
# Player table
unique_players = data["player_name"].unique()

player = pd.DataFrame({"player_id": range(1,len(unique_players)+1), "player_name": unique_players})

# this is wonky but alright
# taken out for easier access; alternative is elsewhere do player.reset_index()
#player.set_index("player_id", inplace=True)

player.to_csv("player.csv")

player.head()


,player_id,player_name
0,1,Meta
1,2,Cracknut
2,3,Edgedancer
3,4,Mailliw73
4,5,Aether


In [144]:
# Game table

# Columns wanted: ID, format, game_num, (game_string), is_anon, broken, IM_id, start_date, 
# end_date, num_cycles, is_sanderson, world, complexity
# TODO get game sheet in an appropriate form for this

# Columns doable from data test: ID, format, game_num, game_string, broken, IM_id

game_parts = data.filter(["format", "game_num", "game_string", "Broken"]).drop_duplicates()

game_parts["game_id"] = range(1, len(game_parts)+1)
#game_parts.set_index("game_id", drop=True, inplace=True)

# need to put IM in as player ID not name
game_IM = data.filter(["game_string", "player_name", "IM"])
game_IM = game_IM.loc[game_IM["IM"] == "Y"] #["player_name"]

#game_IM["IM_id"] = player.loc[player["player_name"] == game_IM["player_name"]]

# really wish player_id stayed int
game_IM = player.merge(game_IM, how="right", on="player_name")

game_partsIM = game_parts.merge(game_IM, how="left", on="game_string")

game_partsIM = game_partsIM.drop(columns=["IM", "player_name"])
game_partsIM = game_partsIM.rename(columns={"player_id": "IM_id"})

game_partsIM.to_csv("game_partial.csv")

game = game_partsIM

game


,format,game_num,game_string,Broken,game_id,IM_id
0,LG,1.0,LG1,NaN,1,NaN
1,LG,2.0,LG2,NaN,2,NaN
2,LG,3.0,LG3,Y,3,NaN
3,LG,4.0,LG4,Y,4,NaN
4,QF,1.0,QF1,NaN,5,NaN
...,...,...,...,...,...,...
204,LG,82.0,LG82,NaN,204,253.0
205,QF,57.0,QF57,NaN,205,236.0
206,MR,55.0,MR55,NaN,206,65.0
207,AG,8.0,AG8,NaN,207,12.0


In [114]:
# GMs table
gm_list = data.loc[data["GM"] == "Y"]

gm_list = gm_list.filter(["player_name", "game_string"]) # format and # come from game table atm

gm_list = player.merge(gm_list, how="right", on="player_name") # ? rename player name/id to gm name/id?

gm_list = game.merge(gm_list, how="right", on="game_string")
gm_list = gm_list.drop(columns=["Broken", "IM_id"]).drop_duplicates()
# drop_duplicates needed because QF38 had two IMs

gm_list.to_csv("gm_partial.csv")

gm_list

,format,game_num,game_string,game_id,player_id,player_name
0,LG,1.0,LG1,0,1,Meta
1,LG,2.0,LG2,1,1,Meta
2,LG,3.0,LG3,2,16,Peng
3,LG,4.0,LG4,3,6,Claincy
4,QF,1.0,QF1,4,49,Tempus
...,...,...,...,...,...,...
293,QF,57.0,QF57,204,335,The Unknown Order
294,MR,55.0,MR55,205,253,Elandera
295,AG,8.0,AG8,206,139,Elbereth
296,AG,8.0,AG8,206,241,Fifth Scholar


In [123]:
data.columns

Index(['ID', 'player_name', 'alignment_type', 'win', 'first_hit', 'last_hit',
       'num_hits', 'death', 'inactive', 'GM', 'IM', 'Spec', 'format',
       'game_num', 'game_string', 'Broken', 'role', 'secondary_role'],
      dtype='object')

In [118]:
# Alignments table
data["alignment_type"].unique()

array([nan, 'G', 'E', 'B', 'F', 'D', 'M', 'N', 'C', 'S'], dtype=object)

In [124]:
# Deaths table
data["death"].unique()

array([nan, 'E', 'X', 'V', 'S', 'F', 'N', 'M', 'I', 'P', 'L?', 'O', 'D',
       '?', 'V/M', 'L', 'V/E'], dtype=object)

In [146]:
# Player-Game table
pg = data.loc[data["GM"] != "Y"]
pg = pg.loc[pg["IM"] != "Y"]
pg = pg.drop(columns=["GM", "IM", "Broken", "ID", "Spec", "format", "game_num"])

pg = player.merge(pg, how="right", on="player_name")
pg = game.merge(pg, how="right", on="game_string").drop(columns=["IM_id", "Broken"])


pg.to_csv("playergame.csv")

pg


,format,game_num,game_string,game_id,player_id,player_name,alignment_type,win,first_hit,last_hit,num_hits,death,inactive,role,secondary_role
0,LG,1.0,LG1,1,2,Cracknut,G,L,1.0,1.0,1.0,E,NaN,Tineye,NaN
1,LG,1.0,LG1,1,3,Edgedancer,G,L,2.0,2.0,1.0,X,NaN,Cop,NaN
2,LG,1.0,LG1,1,4,Mailliw73,G,L,4.0,4.0,1.0,X,NaN,Regular,NaN
3,LG,1.0,LG1,1,5,Aether,E,W,4.0,4.0,1.0,V,NaN,Smoker,NaN
4,LG,1.0,LG1,1,6,Claincy,E,W,NaN,NaN,0.0,S,NaN,Vote Cancel,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5322,QF,58.0,QF58,208,345,Szeth_Pancakes,E,L,2.0,2.0,1.0,X,NaN,NaN,NaN
5323,QF,58.0,QF58,208,269,Karnatheon,N,W,3.0,3.0,1.0,X,NaN,NaN,NaN
5324,QF,58.0,QF58,208,312,Illwei,G,W,NaN,NaN,0.0,S,NaN,NaN,NaN
5325,QF,58.0,QF58,208,322,Dannex,G,W,4.0,4.0,1.0,X,NaN,NaN,NaN


In [141]:
# super optional: specs
specs = data.loc[data["Spec"] == "Y"]
specs = specs.filter(["player_name", "game_string"]).reset_index(drop=True) # again format and num get picked back up from game

# ? maybe should make separate game/player tables to merge with?
specs = game.merge(specs, how="right", on="game_string").drop(columns=["IM_id", "Broken"])
specs = player.merge(specs, how="right", on="player_name")

specs

,player_id,player_name,format,game_num,game_string,game_id
0,18,Windy,LG,1.0,LG1,0
1,19,Weiry,LG,1.0,LG1,0
2,20,Alvron,LG,1.0,LG1,0
3,32,A Joe in the Bush,LG,2.0,LG2,1
4,33,Dalinar,LG,2.0,LG2,1
...,...,...,...,...,...,...
1033,214,xxGaea,MR,42.0,MR42,161
1034,156,Straw,MR,43.0,MR43,165
1035,253,Elandera,MR,43.0,MR43,165
1036,4,Mailliw73,AG,7.0,AG7,179
